In [1]:
%matplotlib inline

import os
from subprocess import call
from glob import glob
import yaml

from tqdm import tqdm_notebook as tqdm

import xarray as xr
import numpy as np

import cftime
from datetime import datetime

USER = os.environ['USER']

## spin up dask cluster

In [2]:
from dask.distributed import Client
from dask_jobqueue import PBSCluster

project = 'NCGD0033'

cluster = PBSCluster(queue='regular',
                     cores = 36,
                     processes = 9,
                     memory = '100GB',          
                     project = project,
                     walltime = '04:00:00',
                     local_directory=f'/glade/scratch/{USER}/dask-tmp')
client = Client(cluster)

/glade/work/mclong/miniconda3/envs/py3/lib/python3.6/site-packages/distributed/bokeh/core.py:56: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


In [3]:
Nnodes = 4
cluster.scale(9*Nnodes)

## what to do?

In [4]:
var = 'O2'
varout = var
isel = {}

mean = 'ann'
n_forecast_lead = 10 # related to expected freq from "mean"

diro = f'/glade/scratch/{USER}/calcs/o2-prediction'

## more advanced options

In [5]:
file_format = 'zarr' # nc or zarr for output

file_out = {'mean': f'{diro}/CESM-DP-LE.{varout}.{mean}.mean.{file_format}',
            'drift': f'{diro}/CESM-DP-LE.{varout}.{mean}.mean.drift.{file_format}',
            'anom': f'{diro}/CESM-DP-LE.{varout}.{mean}.mean.anom.{file_format}'}

first_start_year, last_start_year = 1954, 2015
first_clim_year, last_clim_year = 1964, 2014

dir_dple = '/glade/p_old/decpred/CESM-DPLE'
case_prefix = 'b.e11.BDP.f09_g16'

if not os.path.exists(diro):
    call(['mkdir','-p',diro])

#### Define function for writing output based on file extension (zarr or nc)

In [6]:
def write_output(ds,file_out,attrs={}):
    '''Function to write output:
       - optionally add some file-level attrs
       - switch method based on file extension
       '''
    
    if attrs:
        ds.attrs = attrs
        
    ext = os.path.splitext(file_out)[1]
    if ext == '.nc':
        ds.to_netcdf(file_out)
    elif ext == '.zarr':
        ds.to_zarr(file_out)
    else:
        raise ValueError('Unknown output file extension: {ext}')
        

#### Some helper functions for reading datasets

In [7]:
def time_bound_var(ds):
    tb_name = ''
    if 'bounds' in ds['time'].attrs:
        tb_name = ds['time'].attrs['bounds']
    elif 'time_bound' in ds:
        tb_name = 'time_bound'
    else:
        raise ValueError('No time_bound variable found')
    tb_dim = ds[tb_name].dims[-1]
    return tb_name,tb_dim

def fix_time(ds):
    tb_name,tb_dim = time_bound_var(ds)
    time = cftime.num2date(ds[tb_name].isel(M=0).mean(tb_dim),
                           units = ds.time.attrs['units'],
                           calendar = ds.time.attrs['calendar'])

    ds.time.values = time
    ds = ds.drop([tb_name])
    return ds

# compute drift
Drift is assessed by computing a lead-time dependent climatology

1. Assemble forecast ensemble variable into `start_year x lead_time x lat x lon` array. 
1. Compute ensemble mean
1. Filter start dates outside of the "verification window"
1. Compute mean over start dates as a function of lead time: this is the "drift"
1. Subtract drift from full field


First step, generate a `start_year` coordinate.

In [8]:
S = xr.DataArray(np.arange(first_start_year,last_start_year+1,1,dtype='int32')+1,
                 dims='S',
                 attrs={'long_name':'start year'})
S

<xarray.DataArray (S: 62)>
array([1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966,
       1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978,
       1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016], dtype=int32)
Dimensions without coordinates: S
Attributes:
    long_name:  start year

Generate `lead_time` array

In [9]:
L = xr.DataArray(np.arange(1,n_forecast_lead+1,1,dtype='int32'),
                 dims='L',
                 attrs={'long_name':'forecast lead'})
L

<xarray.DataArray (L: 10)>
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int32)
Dimensions without coordinates: L
Attributes:
    long_name:  forecast lead

## Find the files for each start year

In [10]:
files_by_year = {}
n = np.zeros(len(S),dtype='int32')

for i in range(0,len(S)):
    year = S.values[i]-1
    files_by_year[year] = sorted(glob(f'{dir_dple}/monthly/{var}/{case_prefix}.{year}*.nc'))                              
    n[i] = len(files_by_year[year])
    
#-- ensure that we have the same number of files for each start year
n_ensemble = n[0]
np.testing.assert_equal(n_ensemble,n)

print(f'{n_ensemble} files for {len(S)} start years.')

40 files for 62 start years.


## Assemble forecast ensemble  

Generate a `start_year x lead_time x lat x lon` array

In [11]:
%%time

grid_vars = []
ds_list = []

#-- loop over files for each year
for year,files in tqdm(files_by_year.items()):

    #-- open the datasets
    tmp_file_out = f'/glade/scratch/mclong/tmp/CESM-DPLE-{year:04d}-{var}.nc'
    if os.path.exists(tmp_file_out):
        call(['rm','-f',tmp_file_out])
        
    with xr.open_mfdataset(files,
                           concat_dim='M',
                           decode_times = False, 
                           decode_coords = False) as ds:
        
        #-- store "grid" variables, attributes, and encoding
        if not grid_vars:
            grid_vars = [v for v,da in ds.variables.items() if 'time' not in da.dims]
            ds_grid = ds.drop([v for v in ds.variables if v not in grid_vars]).isel(M=0)
            dsattrs = ds.attrs
            attrs = {v:da.attrs for v,da in ds.variables.items()}
            encoding = {v:{key:val for key,val in da.encoding.items() 
                           if key in ['dtype','_FillValue','missing_value']}
                        for v,da in ds.variables.items()}

        #-- fix time and drop extraneous variables
        ds = fix_time(ds)
        ds = ds.drop([v for v in ds.variables if v not in ds.coords and v != var])

        #-- subset?
        if isel:
            ds = ds.isel(**isel)

        #-- compute appropriate average TODO: ds = calc.compute_{freq}_mean(ds).rename({'time':'L'})
        ds = ds.groupby('time.year').mean('time').isel(year=slice(1,11)).rename({'year':'L'})
        ds['L'] = L 
        for v in ds.variables:
            if v in attrs:
                ds[v].attrs = attrs[v]
            if v in encoding:
                ds[v].encoding = encoding[v]
        
        ds.to_netcdf(tmp_file_out)
        
        ds_list.append(tmp_file_out)
    
#-- assemble into single dataset
#ds = xr.concat(ds_list,dim='S')
ds = xr.open_mfdataset(ds_list,concat_dim='S',decode_times=False,decode_coords=False)
ds['S'] = S

#-- rechunk to more suitable sizes
new_chunks = {'S':1,'L':len(L),'M':n_ensemble}
ds = ds.chunk(new_chunks)

OSError: Timed out trying to connect to 'tcp://10.148.10.15:57156' after 3 s: connect() didn't finish in time

Call `persist` on this dataset to load it into distributed memory.

In [12]:
%%time
ds = ds.persist()
ds.info()

xarray.Dataset {
dimensions:
	L = 10 ;
	M = 40 ;
	lat_aux_grid = 395 ;
	moc_z = 61 ;
	nlat = 384 ;
	nlon = 320 ;
	z_t = 60 ;
	z_t_150m = 15 ;
	z_w = 60 ;
	z_w_bot = 60 ;
	z_w_top = 60 ;

variables:
	float32 z_t(z_t) ;
		z_t:units = centimeters ;
		z_t:long_name = depth from surface to midpoint of layer ;
		z_t:valid_min = 500.0 ;
		z_t:valid_max = 537500.0 ;
		z_t:positive = down ;
	float32 z_w(z_w) ;
		z_w:units = centimeters ;
		z_w:long_name = depth from surface to top of layer ;
		z_w:valid_min = 0.0 ;
		z_w:valid_max = 525000.9375 ;
		z_w:positive = down ;
	float32 moc_z(moc_z) ;
		moc_z:units = centimeters ;
		moc_z:long_name = depth from surface to top of layer ;
		moc_z:valid_min = 0.0 ;
		moc_z:valid_max = 549999.0625 ;
		moc_z:positive = down ;
	float32 z_w_top(z_w_top) ;
		z_w_top:units = centimeters ;
		z_w_top:long_name = depth from surface to top of layer ;
		z_w_top:valid_min = 0.0 ;
		z_w_top:valid_max = 525000.9375 ;
		z_w_top:positive = down ;
	float32 z_w_bot(z_w_bot

### generate a `verification_time` matrix

In [13]:
verification_time = S + 0.5 + L - 1
verification_time

<xarray.DataArray (S: 62, L: 10)>
array([[1955.5, 1956.5, 1957.5, ..., 1962.5, 1963.5, 1964.5],
       [1956.5, 1957.5, 1958.5, ..., 1963.5, 1964.5, 1965.5],
       [1957.5, 1958.5, 1959.5, ..., 1964.5, 1965.5, 1966.5],
       ...,
       [2014.5, 2015.5, 2016.5, ..., 2021.5, 2022.5, 2023.5],
       [2015.5, 2016.5, 2017.5, ..., 2022.5, 2023.5, 2024.5],
       [2016.5, 2017.5, 2018.5, ..., 2023.5, 2024.5, 2025.5]])
Dimensions without coordinates: S, L

### compute ensemble mean across forecast ensemble

In [14]:
dse = ds.mean('M')
dse

<xarray.Dataset>
Dimensions:       (L: 10, lat_aux_grid: 395, moc_z: 61, nlat: 384, nlon: 320, z_t: 60, z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
Coordinates:
  * z_t           (z_t) float32 500.0 1500.0 2500.0 ... 512502.8 537500.0
  * z_w           (z_w) float32 0.0 1000.0 2000.0 ... 500004.7 525000.94
  * moc_z         (moc_z) float32 0.0 1000.0 2000.0 ... 525000.94 549999.06
  * z_w_top       (z_w_top) float32 0.0 1000.0 2000.0 ... 500004.7 525000.94
  * z_w_bot       (z_w_bot) float32 1000.0 2000.0 3000.0 ... 525000.94 549999.06
  * lat_aux_grid  (lat_aux_grid) float32 -79.48815 -78.952896 ... 89.47441 90.0
  * z_t_150m      (z_t_150m) float32 500.0 1500.0 2500.0 ... 13500.0 14500.0
  * L             (L) int32 1 2 3 4 5 6 7 8 9 10
Dimensions without coordinates: nlat, nlon
Data variables:
    O2            (L, z_t, nlat, nlon) float32 dask.array<shape=(10, 60, 384, 320), chunksize=(1, 60, 384, 320)>

### Compute "drift"
Filter start dates outside of the "verification window" 
Compute mean over start dates as a function of lead time: this is the "drift"

In [15]:
drift = dse.where((first_clim_year<verification_time) & 
                  (verification_time<last_clim_year+1) ).mean('S')

for v in drift.variables:
    if v in attrs:
        drift[v].attrs = attrs[v]
    if v in encoding:
        drift[v].encoding = encoding[v]
        
drift

<xarray.Dataset>
Dimensions:       (L: 10, lat_aux_grid: 395, moc_z: 61, nlat: 384, nlon: 320, z_t: 60, z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
Coordinates:
  * L             (L) int64 1 2 3 4 5 6 7 8 9 10
  * z_t           (z_t) float32 500.0 1500.0 2500.0 ... 512502.8 537500.0
  * z_w           (z_w) float32 0.0 1000.0 2000.0 ... 500004.7 525000.94
  * moc_z         (moc_z) float32 0.0 1000.0 2000.0 ... 525000.94 549999.06
  * z_w_top       (z_w_top) float32 0.0 1000.0 2000.0 ... 500004.7 525000.94
  * z_w_bot       (z_w_bot) float32 1000.0 2000.0 3000.0 ... 525000.94 549999.06
  * lat_aux_grid  (lat_aux_grid) float32 -79.48815 -78.952896 ... 89.47441 90.0
  * z_t_150m      (z_t_150m) float32 500.0 1500.0 2500.0 ... 13500.0 14500.0
Dimensions without coordinates: nlat, nlon
Data variables:
    O2            (L, z_t, nlat, nlon) float32 dask.array<shape=(10, 60, 384, 320), chunksize=(1, 60, 384, 320)>

### Compute bias correction

In [16]:
anom = ds - drift

for v in anom.variables:
    if v in attrs:
        anom[v].attrs = attrs[v]
    if v in encoding:
        anom[v].encoding = encoding[v]
anom

<xarray.Dataset>
Dimensions:       (L: 10, M: 40, lat_aux_grid: 395, moc_z: 61, nlat: 384, nlon: 320, z_t: 60, z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
Coordinates:
  * z_t           (z_t) float32 500.0 1500.0 2500.0 ... 512502.8 537500.0
  * z_w           (z_w) float32 0.0 1000.0 2000.0 ... 500004.7 525000.94
  * moc_z         (moc_z) float32 0.0 1000.0 2000.0 ... 525000.94 549999.06
  * z_w_top       (z_w_top) float32 0.0 1000.0 2000.0 ... 500004.7 525000.94
  * z_w_bot       (z_w_bot) float32 1000.0 2000.0 3000.0 ... 525000.94 549999.06
  * lat_aux_grid  (lat_aux_grid) float32 -79.48815 -78.952896 ... 89.47441 90.0
  * z_t_150m      (z_t_150m) float32 500.0 1500.0 2500.0 ... 13500.0 14500.0
  * L             (L) int32 1 2 3 4 5 6 7 8 9 10
Dimensions without coordinates: M, nlat, nlon
Data variables:
    O2            (L, M, z_t, nlat, nlon) float32 dask.array<shape=(10, 40, 60, 384, 320), chunksize=(1, 1, 60, 384, 320)>

### write output to file

In [17]:
dsattrs['history'] = f'created by {USER} on {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'

In [18]:
%%time
if var != varout:
    ds.rename({var:varout},inplace=True)
    
write_output(ds, 
             file_out = file_out['mean'], 
             attrs = dsattrs)

ValueError: path '' contains a group

In [19]:
dsattrs['climatology'] = f'{first_clim_year:d}-{last_clim_year}, computed separately for each lead time'

In [ ]:
%%time
anom.rename({var:'anom'},inplace=True)

write_output(anom, 
             file_out = file_out['anom'], 
             attrs = dsattrs)

In [ ]:
%%time
drift.rename({var:'climo'},inplace=True)

write_output(drift, 
             file_out = file_out['drift'], 
             attrs = dsattrs)